In [23]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
stop_words = stopwords.words('english')
stop_words.append('https')

In [3]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        pass
    
    def fit(self, data, y = 0):
        
        return self
    
    def transform(self, data, y = 0):
        normalized_corpus = data.apply(self.process_doc)
        
        return normalized_corpus
        
    def process_doc(self, doc):

        wnl = WordNetLemmatizer()
        stop_words = stopwords.words('english')
        
        def pos_tagger(nltk_tag):
            if nltk_tag.startswith('J'):
                return wordnet.ADJ
            elif nltk_tag.startswith('V'):
                return wordnet.VERB
            elif nltk_tag.startswith('N'):
                return wordnet.NOUN
            elif nltk_tag.startswith('R'):
                return wordnet.ADV
            else:         
                return None

        doc_norm = [tok.lower() for tok in word_tokenize(doc) if ((tok.isalpha()) & (tok.lower() not in stop_words)) ]

        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(doc_norm))) 
        doc_norm = [wnl.lemmatize(token, pos) for token, pos in wordnet_tagged if pos is not None]

        return " ".join(doc_norm)

In [40]:
train = pd.read_csv('sent_train.csv')
test = pd.read_csv('sent_valid.csv')
X_train = train['text']
X_test = test['text']
y_train = train['label']
y_test = test['label']

In [41]:
processor = TextPreprocessor()
X_train_transformed = processor.fit_transform(X_train)
X_train_transformed

0                bynd jpmorgan reel expectation meat http
1       ccl rcl nomura point booking weakness carnival...
2       cx cemex cut credit suisse morgan weak buildin...
3                       es btig research cut neutral http
4                    fnko funko slide jaffray pt cut http
                              ...                        
9538    week gainer loser stoxx europe economy markets...
9539    tupperware brand consumer gainer unilever lead...
9540    vtv therapeutic lead healthcare gainer myomo b...
9541                         work xpo pyx amkr hour mover
9542                              yndx qd oesx tech mover
Name: text, Length: 9543, dtype: object

In [34]:
steps = [('vec', CountVectorizer()), ('ros', RandomOverSampler()), ('MultiNB', MultinomialNB())]
model_pipe = Pipeline(steps)

In [35]:
model_pipe

Pipeline(steps=[('ros', RandomOverSampler()), ('vec', CountVectorizer()),
                ('MultiNB', MultinomialNB())])

In [43]:
model_pipe.fit(X_train_transformed, y_train)

ValueError: Expected 2D array, got 1D array instead:
array=['bynd jpmorgan reel expectation meat http'
 'ccl rcl nomura point booking weakness carnival royal caribbean http'
 'cx cemex cut credit suisse morgan weak building outlook http' ...
 'vtv therapeutic lead healthcare gainer myomo biohaven pharmaceutical loser'
 'work xpo pyx amkr hour mover' 'yndx qd oesx tech mover'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.